In [24]:
import json
import numpy as np
from collections import Counter
from matplotlib import pyplot as plt
from utils import *
from tqdm import tqdm, trange

In [2]:
data = [json.loads(line) for line in open('datasets/MovieLens-1M/data.jsonl')]
id2name = json.load(open('datasets/MovieLens-1M/id2name.json'))

In [9]:
outputs = np.array([json.loads(line) for line in open('zero-shot-dumps/gpt2_pattern1_batchify.jsonl')][:750])
top_preds = outputs.argsort(-1)
labels = np.array([item['ids'][:7][-2] for item in data[:len(top_preds)]])
compute_recall_at_k(top_preds, labels)

0.06666666666666667

In [10]:
calibration = np.array(json.load(open('zero-shot-dumps/gpt2_pattern1_calibration.json')))

In [13]:
top_preds

array([[ 740, 3370, 1176, ..., 3011,  684, 3338],
       [ 740, 1192, 1178, ..., 1140,  684, 3011],
       [ 740, 3370, 1176, ...,  684, 3011, 3338],
       ...,
       [ 740, 1192, 1178, ..., 1140,  684, 3011],
       [ 740, 1176, 3370, ..., 3338,  684, 3011],
       [ 740, 3370, 1176, ...,  764, 3011,  684]])

In [15]:
top_pred = calibration.argsort(-1)
top_pred

array([ 740, 3370, 1176, ..., 3338, 1440,  764])

In [23]:
from scipy import stats
corrs = [stats.spearmanr(output, calibration)[0] for output in outputs]
print(np.mean(corrs), np.std(corrs))

0.8289260171195261 0.054541750341739206


In [5]:
[id2name[i] for i in np.argsort(np.array(calibration))[:5]]

['Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb',
 'Teenage Mutant Ninja Turtles II: The Secret of the Ooze',
 "One Flew Over the Cuckoo's Nest",
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode VI - Return of the Jedi']

In [34]:
rs = []
outputs = np.log(np.array([json.loads(line) for line in open('zero-shot-dumps/gpt2_pattern1_batchify.jsonl')])[:750])
calibration = np.log(calibration).reshape(1, -1)
labels = np.array([item['ids'][:7][-2] for item in data[:len(top_preds)]])
for alpha in trange(100):
    scores = outputs - 0.01 * alpha * calibration
    top_preds = np.array(scores).argsort(-1)
    r = compute_recall_at_k(top_preds, labels)
    rs.append(r)

100%|██████████| 100/100 [00:20<00:00,  4.79it/s]


In [37]:
np.argmax(rs)

40

In [52]:
scores = outputs - 0.4 * calibration
top_preds = np.array(scores).argsort(-1)
compute_recall_at_k(top_preds, labels)

0.07333333333333333

In [53]:
top_preds

array([[3370,  740, 1176, ..., 3011,  684, 3338],
       [1192, 1178, 2559, ..., 1140,  684, 3011],
       [3370,  589, 1176, ..., 3338,  684, 3011],
       ...,
       [1985,  326, 1081, ..., 1140,  684, 3011],
       [1230, 1081, 3402, ..., 2754,  684, 3011],
       [ 740, 3370, 1176, ..., 1092, 3011,  684]])

In [43]:
def pattern(ids, id2name):
    s = 'A user watched '
    for id in ids:
        s += id2name[id] + ', '
    s = s.strip()[:-1] + '. '
    s += 'Now the user may want to watch'
    return s

In [51]:
idx = 7
print(idx)
print()
ids, label, top_pred, score = data[idx]['ids'][:7][:-2], data[idx]['ids'][:7][-2], list(top_preds[idx]), scores[idx]
print(pattern(ids, id2name))
print()
print('\n'.join([str((id2name[pred], score[pred])) for pred in top_pred[:5]]))
print()
print('\n'.join([str((id2name[pred], score[pred])) for pred in top_pred[-5:]]))
print()
print(id2name[label], top_pred.index(label))

7

A user watched Star Wars: Episode VI - Return of the Jedi, Taxi Driver, North by Northwest, High Fidelity, Face/Off. Now the user may want to watch

('Under the Domin Tree (Etz Hadomim Tafus)', -10.72523526123218)
('Goodbye, 20th Century (Zbogum na dvadesetiot vek)', -10.679579765576529)
('Les convoyeurs attendent', -10.669626414598188)
('Wend Kuuni', -10.337777999400679)
('Salut cousin!', -10.080008973209521)

("One Flew Over the Cuckoo's Nest", -2.2180148709238123)
('Star Wars: Episode I - The Phantom Menace', -2.1884219089586296)
('Star Wars: Episode VI - Return of the Jedi', -2.1820252418997304)
('Teenage Mutant Ninja Turtles II: The Secret of the Ooze', -2.157260324959569)
('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', -1.5006682302995564)

Jurassic Park 2759
